In [1]:
import importlib
from typing import List, Tuple
import pandas as pd
import numpy as np

from constants import *
import utils
importlib.reload(utils)

<module 'utils' from '/mnt/cb8c2a81-32e7-40bc-af74-0c1386cda683/programowanie/pipelon/Classifier-Builder/lol_classifier/utils.py'>

In [2]:
from glob import glob
import os

datasets = glob(os.path.join(DATASET_PATH, f"*/{DATASET_POSTFIX}"))
print(f"Found {len(datasets)} datasets")

Found 9 datasets


In [16]:
utils.clear_directories([TRAINING_DATA_PATH, TESTING_DATA_PATH])
for path in datasets:
    filename = path.split('/')[-2]

    df = utils.read_eeg_data(path)
    df.drop(df[df.iloc[:, -1] == 'd'].index, inplace=True)  # drop flashes
    INITIAL_MARGIN = SAMPLING_RATE * 10
    df.iloc[INITIAL_MARGIN:SAMPLING_RATE * 25, -1] = 'neutral'

    # Apply notch filter using the utility function
    # df.iloc[:, :-1] = utils.apply_notch_filter(df.iloc[:, :-1], sampling_rate=SAMPLING_RATE, freqs=8.0)

    # Apply band-pass filter using the utility function
    # df.iloc[:, :-1] = utils.apply_bandpass_filter(df.iloc[:, :-1], sampling_rate=SAMPLING_RATE, l_freq=1.0, h_freq=10.0)

    markers = df.iloc[:, -1]
    print(f"From {filename} with shape:{df.shape} and markers:{len(markers[markers != '0'])} kills:{len(markers[markers == 'kill'])}, deaths:{len(markers[markers == 'death'])}")

    prev_marker_and_countdown = ['0', 0]
    df.iloc[:, -1] = markers.apply(lambda x: utils.propagate_events(x, prev_marker_and_countdown))

    df.drop(df[markers == '0'].index, inplace=True)  
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    utils.split_and_save(df, f"{filename}.csv")


From lol_2024-08-04T21-10-29 with shape:(41331, 41) and markers:382 kills:0, deaths:7
From lol_2024-08-08T14-29-23 with shape:(61232, 41) and markers:384 kills:0, deaths:9
From lol_2024-08-08T16-38-51 with shape:(49694, 41) and markers:395 kills:10, deaths:10
From lol_2024-08-08T19-40-51 with shape:(44081, 41) and markers:390 kills:8, deaths:7
From lol_2024-08-08T20-58-04 with shape:(30700, 41) and markers:386 kills:3, deaths:8
From lol_2024-08-09T14-47-50 with shape:(54396, 41) and markers:391 kills:8, deaths:8
From lol_2024-08-12T11-51-55 with shape:(49782, 41) and markers:384 kills:5, deaths:4
From lol_2024-08-12T15-18-29 with shape:(53692, 41) and markers:384 kills:0, deaths:9
From lol_2024-08-12T17-29-12 with shape:(42603, 41) and markers:387 kills:8, deaths:4


In [18]:
train, test = utils.load_datasets()
train.shape, test.shape 

Number of kills=2477; deaths=3879; neutral=3231


((9587, 41), (2401, 41))

In [19]:
train[np.isnan(train).any(axis=1)]


array([], shape=(0, 41), dtype=float64)

In [20]:
from sklearn.svm import SVC

X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

svm = SVC(kernel='poly', degree=3, C=1, random_state=42, decision_function_shape='ovr', probability=True)
svm.fit(X_train, y_train)

SVC(C=1, kernel='poly', probability=True, random_state=42)

In [21]:
print('accuracy:', svm.score(X_test, y_test))

from sklearn.metrics import classification_report, confusion_matrix
y_pred = svm.predict(X_test)

utils.present_confusion_matrix(confusion_matrix(y_test, y_pred))
utils.present_metrics(classification_report(y_test, y_pred, target_names=STATES, output_dict=True))

accuracy: 0.8900458142440649
Confusion Matrix:
+-----------------+-----------------+------------------+--------------------+
|                 | Predicted: kill | Predicted: death | Predicted: neutral |
+-----------------+-----------------+------------------+--------------------+
|  Actual: kill   |       501       |       104        |         15         |
|  Actual: death  |       98        |       857        |         16         |
| Actual: neutral |        9        |        22        |        779         |
+-----------------+-----------------+------------------+--------------------+

Classification Report:
+--------------+-----------+--------+----------+---------+
|              | precision | recall | f1-score | support |
+--------------+-----------+--------+----------+---------+
|     kill     |   0.824   | 0.808  |  0.816   |  620.0  |
|    death     |   0.872   | 0.883  |  0.877   |  971.0  |
|   neutral    |   0.962   | 0.962  |  0.962   |  810.0  |
|   accuracy   |   0.89    | 